In [1]:
!pip install bertopic
import os
from google.colab import drive
from bertopic import BERTopic
drive.mount('/content/drive/')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 76 kB 3.2 MB/s 
     |████████████████████████████████| 636 kB 11.3 MB/s 
     |████████████████████████████████| 5.2 MB 59.5 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 85 kB 2.8 MB/s 
     |████████████████████████████████| 88 kB 9.0 MB/s 
     |████████████████████████████████| 4.7 MB 37.5 MB/s 
     |████████████████████████████████| 1.3 MB 57.4 MB/s 
     |████████████████████████████████| 120 kB 61.5 MB/s 
     |████████████████████████████████| 6.6 MB 46.6 MB/s 
     |████████████████████████████████| 1.1 MB 63.0 MB/s 
  Created wheel for hdbscan: filename=hdbscan-0.8.28-cp37-cp37m-linux_x86_64.whl size=2340298 sha256=550f014eebddbc56edb220d354185ab3510dbf58fd3677a916c6481dad4d4b6a
  Stored in directory: /root/.cache

In [2]:
cd /content/drive/MyDrive

/content/drive/MyDrive


In [3]:

#전처리 패키지
import pandas as pd
import numpy as np
from datetime import datetime
import re
from nltk.corpus import stopwords
from sklearn.feature_extraction._stop_words import ENGLISH_STOP_WORDS
import spacy
import string

#토픽 모델링 패키지
from bertopic import BERTopic
import tensorflow_hub
from umap import UMAP
import hdbscan
import sklearn
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

#그 외
import warnings


In [4]:
#MyDrive data경로에 뉴스 파일을 업로드 해줘야함
cnbc=pd.read_csv('./data/US_topics_tagged.csv')

In [5]:
#중복제거 후 데이터 생성
#BERTopic의 경우 텍스트 전처리를 사전에 진행하는 것보다 후에 처리하는 것이 더 좋은 결과를 뽑아낼 수 있습니다.

filter_cnbc=cnbc.drop_duplicates(['date','contents'],keep='first') # 중복 제거 

filter_cnbc['date']=filter_cnbc.date.astype(str) 
filter_cnbc['date']=filter_cnbc['date'].apply(lambda x : x[:8]) 
filter_cnbc['date']=pd.to_datetime(filter_cnbc['date']) #datetime 변환

filter_cnbc[filter_cnbc['contents']=='nan'] = np.nan
filter_cnbc.dropna(subset=['contents'],how='any',axis=0,inplace=True) #결측치 제거

#filter_cnbc.contents=filter_cnbc.contents.apply(lambda x : text_preprocessing(x))    

week_cnbc=filter_cnbc[filter_cnbc.date.between('2022-08-18','2022-08-29')].sort_values('date') #원하는 날짜 조정 (주간 단위)
week_CNBC=week_cnbc.reset_index(drop=True)



In [6]:
#뉴스 문서 테마 라벨링을 위한 sub_news data생성
cnbc['date']=cnbc.date.astype(str) 
cnbc['date']=cnbc['date'].apply(lambda x : x[:8]) 
cnbc['date']=pd.to_datetime(cnbc['date']) #datetime 변환

cnbc[cnbc['contents']=='nan'] = np.nan
cnbc.dropna(subset=['contents'],how='any',axis=0,inplace=True) #결측치 제거

#cnbc.contents=cnbc.contents.apply(lambda x : text_preprocessing(x))   

all_week_cnbc=cnbc[cnbc.date.between('2022-08-18','2022-08-29')].sort_values('date') #원하는 날짜 조정 (주간 단위)
all_week_cnbc=all_week_cnbc.reset_index(drop=True)



In [7]:
#형태소 분석 모델 다운로드
!python -m spacy download en_core_web_lg
#!python -m spacy download en_core_web_trf

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 587.7 MB 9.8 kB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [10]:
#형태소 분석 모델 정의 (spacy)
nlp = spacy.load("en_core_web_lg")#trf 모델로 변경가능
spacy_stopwords_list = list(nlp.Defaults.stop_words)
nlp.max_length = 40000000

nlp.get_pipe('attribute_ruler').add([[{"TEXT":"Russias"}]],{"LEMMA":"Russia"}) #굳이 안써도 됨.
nlp.get_pipe('attribute_ruler').add([[{"TEXT":"Ukraines"}]],{"LEMMA":"Ukraine"})
nlp.get_pipe('attribute_ruler').add([[{"TEXT":"Russians"}]],{"LEMMA":"Russian"})
nlp.get_pipe('attribute_ruler').add([[{"TEXT":"Ukrainians"}]],{"LEMMA":"Ukrainian"})

#추가 불용어리스트 정의하고자 하면 사용
stop_words_list=["year","u","cnbc","cnbcs","investor","s","t","p","m","c","k","cramer","cramers","jim","inc","q","fy","b","yahoo","bloomberg"]
for word in stop_words_list:
    spacy_stopwords_list.append(word)

In [11]:
#처음 토픽을 찾을 때 활용하긴 적합하지 않고 토픽을 뽑아낸 후 한번더 거르고 싶을 때 사용하는 것이 좋음
seed_topic_list=[['earning','report','share'], ['twitter','subponae','whistleblower'],
              ['covid-19','astrazeneca','fda'], ['oil','berkshire','occidental'],
              ['union','starbucks','ge'],['amazon','signify','healthcare','medical'],['ad','apple','meta'],['abortion','walmart','travel']]

In [12]:
#vectorizer_model = CountVectorizer(stop_words=stopwords) 
#vectorizer_model = TfidfVectorizer(max_df=0.9)# countvectorizer과 선택해서 사용

umap_model = UMAP(n_neighbors=10, min_dist=0.1, n_components=4,random_state=42,metric='cosine') #umap_model 정의 
hdbscan_model=hdbscan.HDBSCAN(min_cluster_size=4, metric='euclidean',cluster_selection_method='eom', prediction_data=True)#hdbscan_model 정의

embedding_model = tensorflow_hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")
# nr_topics ='auto' auto 제거시 더 많은 토픽을 볼 수 있음 (auto시 자동으로 유사도가 높은 토픽끼리 합침)
topic_model = BERTopic(verbose=True, embedding_model=embedding_model,top_n_words=15,min_topic_size=5,umap_model=umap_model,hdbscan_model=hdbscan_model,seed_topic_list=seed_topic_list)
#데이터 좀 많을 떄 min_topic_size=10 , min_cluster_Size=6 등으로 조절

In [13]:
# False로 할시 속도 향상
topic_model.calculate_probabilities = True  

In [14]:
topics, probabillities= topic_model.fit_transform(week_CNBC.contents)

  0%|          | 0/489 [00:00<?, ?it/s]WARNING:tensorflow:From /usr/local/lib/python3.7/dist-packages/bertopic/backend/_use.py:53: _EagerTensorBase.cpu (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.identity instead.
100%|██████████| 489/489 [00:03<00:00, 133.94it/s]
2022-08-30 06:56:14,799 - BERTopic - Transformed documents to Embeddings
100%|██████████| 8/8 [00:00<00:00, 177.95it/s]
2022-08-30 06:56:22,907 - BERTopic - Reduced dimensionality
2022-08-30 06:56:22,982 - BERTopic - Clustered reduced embeddings


In [15]:
#(spacy) tag_ N으로 시작하고 정의한 불용어 사전에 포함되지 않고 entity에서 TIME, CARDINAL, DATE, PERSON은 제거 
#그리고 소문자로 변환해서 조건을 만족하는 값만 반환
#output을 뽑아낸 topics 변수에서 사후 텍스트 전처리 진행 
def tokenizer(text):
    words=[]
    doc=nlp(text)
    for token in doc:
        if token.tag_[0] in ['N','V'] and token.lemma_.lower() not in ENGLISH_STOP_WORDS and token.ent_type_ not in ['TIME','CARDINAL','DATE','PERSON']:
                if len(token.lemma_.lower())>1:
                    words.append(token.lemma_.lower())
                
    return words


In [17]:
vectorizer_model = TfidfVectorizer(tokenizer=tokenizer,stop_words="english", ngram_range=(1, 2),min_df=2)
topic_model.update_topics(week_CNBC.contents, topics, vectorizer_model=vectorizer_model)#위에서 정의한 사용자 정의 tokenzier를 활용해 불용어 제거

In [18]:
topic_model.get_topic_info() #전체 정보

,Topic,Count,Name
0,-1,93,-1_say_auction_card_sap
1,0,44,0_earning_share_revenue_zoom
2,1,28,1_vehicle_tesla_tax credit_car
3,2,21,2_inflation_rate_wage_fed
4,3,20,3_vaccine_virus_monkeypox_dose
5,4,19,4_alibaba_china_singapore_pinduoduo
6,5,18,5_taiwan_apple_china_iphone
7,6,16,6_berkshire_occidental_buyback_gm
8,7,15,7_streaming_hbo_sport_amazon
9,8,15,8_ukraine_russia_plant_iran


In [19]:
topic_model.get_topic(25)# 토픽 하나씩 뽑아서 키워드 체크

[('twitter', 0.07590634095285827),
 ('zatko', 0.0602195524861681),
 ('whistleblower', 0.050417957798227594),
 ('security', 0.03635134402707356),
 ('complaint', 0.03523625650617009),
 ('allege', 0.027821703967005472),
 ('musk', 0.02778177938022764),
 ('spam', 0.02380474560921207),
 ('cnn', 0.02308572421923483),
 ('cybersecurity', 0.020927927443958237),
 ('decree', 0.020779666052428567),
 ('datum', 0.020098804275736126),
 ('mudge', 0.019945636159329087),
 ('news organization', 0.0198015265637746),
 ('breach', 0.01974257697663143)]

In [20]:
similar_topics, similarity = topic_model.find_topics("moderna", top_n=5); similar_topics # 특정단어와 가장 연관성 높은 토픽 찾기 

[12, 9, 31, 27, 25]

In [21]:
topic_model.get_representative_docs(14)[2] # 토픽에 속한 문서 내용 최대 3개까지 볼 수 잇음

'J M Smucker Co\xa0(NYSE: SJM) reported\xa0first-quarter FY23 sales growth of 1%\xa0year-on-year to $1.873 billion, marginally above the consensus. Net sales excluding divestitures and foreign currency exchange increased 4%. Adjusted EPS of $1.67 beat the analyst consensus of $1.27. Gross profit decreased 14% Y/Y to $552.5 million and the gross margin contracted by 490 basis points to 29.5%. The operating margin contracted 440 basis points to 9.6%, and operating income for the quarter fell 31% to $179.7 million. The company held $151.6 million in cash and equivalents as of July 31, 2022. Cash used in operating activities totaled $(39) million with a free cash flow of $(127.3) million. Outlook: J M Smucker raised FY23 adjusted EPS guidance to $8.20 - $8.60 from $7.85 - $8.25 (consensus $8.04). It sees net sales increase to be 4% - 5% from the previous outlook of 3.5% - 4.5%.'

In [22]:
#bertopic 새로운 기능 hierarchical_topics를 활용해 비슷한 topic끼리는 묶어주자!
from scipy.cluster import hierarchy as sch

In [23]:
linkage_function = lambda x: sch.linkage(x, 'single', optimal_ordering=True)

In [24]:
hierarchical_topics = topic_model.hierarchical_topics(week_CNBC.contents, topics,linkage_function=linkage_function)

100%|██████████| 32/32 [00:00<00:00, 193.09it/s]


In [25]:
tree = topic_model.get_topic_tree(hierarchical_topics)

In [26]:
print(tree)

.
├─■──vaccine_virus_monkeypox_dose_fda ── Topic: 3
└─say_company_share_market_stock
     ├─say_company_share_market_twitter
     │    ├─twitter_abortion_court_election_musk
     │    │    ├─twitter_abortion_election_musk_trump
     │    │    │    ├─twitter_abortion_election_musk_trump
     │    │    │    │    ├─twitter_musk_election_trump_document
     │    │    │    │    │    ├─twitter_musk_zatko_spam_whistleblower
     │    │    │    │    │    │    ├─■──musk_twitter_subpoena_dorsey_spam ── Topic: 28
     │    │    │    │    │    │    └─■──twitter_zatko_whistleblower_security_complaint ── Topic: 25
     │    │    │    │    │    └─election_trump_pakistan_campaign_affidavit
     │    │    │    │    │         ├─trump_affidavit_election_raid_campaign
     │    │    │    │    │         │    ├─■──affidavit_trump_raid_warrant_fbi ── Topic: 18
     │    │    │    │    │         │    └─■──campaign_donor_trump_primary_house ── Topic: 20
     │    │    │    │    │         └─■──pakistan_election

In [27]:
topic_model.visualize_hierarchy(hierarchical_topics=hierarchical_topics)

In [28]:
#토픽 merge
topics_to_merge = [28, 25]
topic_model.merge_topics(week_CNBC.contents, topics, topics_to_merge)

In [29]:
#32,33번 토픽인 트위터가 하나로 합쳐진 것을 볼 수 있다.
topic_model.get_topic_info() #전체 정보

,Topic,Count,Name
0,-1,93,-1_say_auction_card_sap
1,0,44,0_earning_share_revenue_zoom
2,1,28,1_vehicle_tesla_tax credit_car
3,2,21,2_inflation_rate_wage_fed
4,3,20,3_vaccine_virus_monkeypox_dose
5,4,19,4_alibaba_china_singapore_pinduoduo
6,5,18,5_taiwan_apple_china_iphone
7,6,16,6_berkshire_occidental_buyback_gm
8,7,15,7_streaming_hbo_sport_amazon
9,8,15,8_ukraine_russia_plant_iran


In [ ]:
#한번에 여러개 묶고 싶을 때 사용
topics_to_merge = [[1, 2]
                   [3, 4]]
topic_model.merge_topics(docs, topics, topics_to_merge)

In [ ]:
#calculate_probabilities =True 로 계산시 활용가능 -1로 분류된 문서들을 가장 연관있는 토픽으로 재분류
import numpy as np
probability_threshold = 0.01
new_topics = [np.argmax(prob) if max(prob) >= probability_threshold else -1 for prob in probabillities]

In [ ]:
week_CNBC['topics']=new_topics
topic_df=topic_model.get_topic_info() #전체 정보

In [ ]:
small_cnbc=week_CNBC.loc[:,['link','contents','topics']]

In [ ]:
topic_first=pd.merge(all_week_cnbc,small_cnbc,how='left',left_on=['link','contents'],right_on=['link','contents'])

In [ ]:
last=pd.merge(topic_first,topic_df,how='left',left_on='topics',right_on='Topic')

In [ ]:
save=pd.merge(week_CNBC,topic_df,how='left',left_on='topics',right_on='Topic')

In [ ]:
last.to_csv('0825_topics.csv',encoding='utf-8-sig')